In [1]:
import utils
import anndata
import numpy as np
import pandas as pd
import requests
import scanpy as sc
import scanpy.external as sce
import scanorama
import os
import anndata
from typing import List, Optional
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
import umap
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans
import plotly.express as px
import plotly.io as pio
import src.vanilla_cvae as vanilla_cvae
import src.gma_cvae as gma_cvae
import seaborn as sns


In [2]:
def plot_pca_2d(data, labels):
    """Generate 2D PCA plot of the data"""
    print("Generating 2D PCA plot...")
    
    # Perform PCA
    pca = PCA(n_components=2)
    pca_result = pca.fit_transform(data)
    
    # Create a DataFrame for plotting
    pca_df = pd.DataFrame(pca_result, columns=['PCA Component 1', 'PCA Component 2'])
    pca_df['Dataset'] = labels
    
    # Plotting with Plotly
    fig = px.scatter(
        pca_df, x='PCA Component 1', y='PCA Component 2',
        color='Dataset',
        labels={'PCA Component 1': 'PCA Component 1', 'PCA Component 2': 'PCA Component 2'},
        title='PCA: Generated Data'
    )
    fig.update_traces(marker=dict(size=2.5))
    #fig.write_html('pca_2d_plot.html')
    fig.show()

In [3]:
path = os.getcwd()[:-3]
batch_corrected  = sc.read_h5ad(path + 'data/corrected_data.h5ad')
unbatch_corrected = sc.read_h5ad(path + 'data/unbatch_corrected_data.h5ad')

C:\Creations\Research\NASASLSTP25\AI4LS\AI4LS\CVAE\.venv_cvae\Lib\site-packages\anndata\_core\anndata.py:1756: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
C:\Creations\Research\NASASLSTP25\AI4LS\AI4LS\CVAE\.venv_cvae\Lib\site-packages\anndata\_core\anndata.py:1756: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [5]:
batch_corrected.obs = batch_corrected.obs.drop(columns=['dataset'])
sc.pp.highly_variable_genes(batch_corrected, n_top_genes=2000)
adata = batch_corrected[:, batch_corrected.var['highly_variable']]

In [6]:
unbatch_corrected.obs = unbatch_corrected.obs.drop(columns=['dataset'])
sc.pp.highly_variable_genes(unbatch_corrected, n_top_genes=2000)
unbatch_corrected = unbatch_corrected[:, unbatch_corrected.var['highly_variable']]

In [7]:
combined_data = np.vstack([batch_corrected, unbatch_corrected])
labels = np.array(['Batch Corrected'] * batch_corrected.shape[0] + ['UnBatch Corrrected'] * unbatch_corrected.shape[0])
plot_pca_2d(combined_data, labels)

KeyboardInterrupt: 